In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt 
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
    
import h5py
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
try:
    import dill as pickle
except ImportError:
    import pickle

# Loading Precomputed Data

In [4]:
# Loading Image Features
#train_features = load(open("./drive/My Drive/Project/theme_encoded_train_images.pkl", "rb"))
train_features = load(open("./drive/My Drive/Project/urdu/urdu_encoded_train_images.pkl", "rb"))
#train_features = load(open("./drive/My Drive/Project/urdu/urdu_encoded_train_images (1).pkl", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=500


In [5]:
# Load Train Descriptions, which contains, 5 captions corresponding to a key
#train_descriptions = load(open("./drive/My Drive/Project/theme_train_descriptions.pkl", "rb"))
train_descriptions = load(open("./drive/My Drive/Project/urdu/urdu_train_descriptions.pkl", "rb"))
print('Photos: train=%d' % len(train_descriptions))

Photos: train=500


In [6]:
# Loading embedding_matrix
#embedding_matrix = load(open("./drive/My Drive/Project/theme_Embedding_Matrix.pkl", "rb"))
embedding_matrix = load(open("./drive/My Drive/Project/urdu/urdu_Embedding_Matrix.pkl", "rb"))
print('Embedding Matrix shape = (%d, %d)' % (embedding_matrix.shape))

Embedding Matrix shape = (369, 300)


In [7]:
# Loading wordtoix
#wordtoix = load(open("./drive/My Drive/Project/theme_wordtoix.pkl", "rb"))
wordtoix = load(open("./drive/My Drive/Project/urdu/urdu_wordtoix.pkl", "rb"))
print('Wordtoix Length = %d' % len(wordtoix))

Wordtoix Length = 368


# Data Generator

In [8]:
max_length = 34 # computer from file 2
epochs = 100
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [9]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0
    return X1, X2, y

In [10]:
max_length = 35 # computer from file 2
epochs = 100
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [11]:
#train_features['2513260012_03d33305cf.jpg'].shape

# Defining the Model

In [12]:
max_length = 35 
vocab_size = embedding_matrix.shape[0]
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath
embedding_dim = embedding_matrix.shape[1]
max_length,vocab_size ,embedding_dim

(35, 369, 300)

In [13]:
inputs1 = Input(shape=(2048,))
#inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
my_model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [14]:
my_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 35)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 35, 300)      110700      input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
____________________________________________________________________________________________

In [15]:
my_model.layers[2]

In [16]:
my_model.layers[2].set_weights([embedding_matrix])
my_model.layers[2].trainable = False

In [17]:
my_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0004))

In [18]:
#my_model.save('model_' + 'test' + '.h5')
#my_model.save('theme_model_' + 'test' + '.h5')
my_model.save('urdu_model_' + 'test' + '.h5')

In [19]:
epochs = 60
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [20]:
cnt=1
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    my_model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    my_model.save('model_' + str(i) + '.h5')
    print(cnt)
    cnt+=1

Epoch 1/1
83/83 [==============================] - 20s 237ms/step - loss: 4.4351
1
Epoch 1/1
83/83 [==============================] - 19s 227ms/step - loss: 3.7194
2
Epoch 1/1
83/83 [==============================] - 19s 231ms/step - loss: 3.3372
3
Epoch 1/1
83/83 [==============================] - 19s 228ms/step - loss: 3.1163
4
Epoch 1/1
83/83 [==============================] - 19s 229ms/step - loss: 2.9383
5
Epoch 1/1
83/83 [==============================] - 19s 224ms/step - loss: 2.8316
6
Epoch 1/1
83/83 [==============================] - 19s 227ms/step - loss: 2.7220
7
Epoch 1/1
83/83 [==============================] - 19s 226ms/step - loss: 2.6367
8
Epoch 1/1
83/83 [==============================] - 19s 226ms/step - loss: 2.5437
9
Epoch 1/1
83/83 [==============================] - 19s 228ms/step - loss: 2.4750
10
Epoch 1/1
83/83 [==============================] - 19s 226ms/step - loss: 2.3885
11
Epoch 1/1
83/83 [==============================] - 19s 225ms/step - loss: 2.3116
12
E

In [21]:
#with open('Caption_generator_Model.pkl', 'wb') as model:
    #pickle.dump(my_model, model)
#with open('theme_Caption_generator_Model.pkl', 'wb') as model:
with open('urdu_Caption_generator_Model.pkl', 'wb') as model:
    pickle.dump(my_model, model)